# Load the trained model and freeze it
For the model to be optimized using TensorRT for specific hardware it must be frozen.
The required code is taken from the [leimao/Frozen_Graph_TensorFlow](https://github.com/leimao/Frozen_Graph_TensorFlow/tree/master/TensorFlow_v2) GitHub repository.

In [2]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [3]:
model = tf.keras.models.load_model('saved_model/mobilenetv2')

In [4]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           Input[0][0]                      
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [5]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [6]:
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
#frozen_func.graph.as_graph_def()

In [7]:
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

--------------------------------------------------
Frozen model layers: 
x
model_2/Conv1_pad/Pad/paddings
model_2/Conv1_pad/Pad
model_2/Conv1/Conv2D/ReadVariableOp/resource
model_2/Conv1/Conv2D/ReadVariableOp
model_2/Conv1/Conv2D
model_2/bn_Conv1/ReadVariableOp/resource
model_2/bn_Conv1/ReadVariableOp
model_2/bn_Conv1/ReadVariableOp_1/resource
model_2/bn_Conv1/ReadVariableOp_1
model_2/bn_Conv1/FusedBatchNormV3/ReadVariableOp/resource
model_2/bn_Conv1/FusedBatchNormV3/ReadVariableOp
model_2/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1/resource
model_2/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1
model_2/bn_Conv1/FusedBatchNormV3
model_2/Conv1_relu/Relu6
model_2/expanded_conv_depthwise/depthwise/ReadVariableOp/resource
model_2/expanded_conv_depthwise/depthwise/ReadVariableOp
model_2/expanded_conv_depthwise/depthwise
model_2/expanded_conv_depthwise_BN/ReadVariableOp/resource
model_2/expanded_conv_depthwise_BN/ReadVariableOp
model_2/expanded_conv_depthwise_BN/ReadVariableOp_1/resource
model_2/

In [8]:
# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_model",
                  name="frozen_graph.pb",
                  as_text=False)

'./frozen_model/frozen_graph.pb'